# RGCN Experiments

In [1]:
import os
import sys

import numpy as np
import pandas as pd

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import torch
import torch_geometric as pyg
from torch_geometric.nn import RGCNConv, Linear
from torch.nn import functional as F

import src.temporal_loader_v2 as tl
from src.utils import to_homogeneous, score

In [2]:
cuda_device = 4

if torch.cuda.is_available():
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    torch.cuda.set_device(cuda_device)
    
torch.manual_seed(42)

### Model

In [3]:
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch_geometric.nn import RGCNConv, Linear


class RGCN(torch.nn.Module):
    def __init__(self, num_features, dim=16, num_classes=2, num_layers=2):
        super(RGCN, self).__init__()

        self.conv1 = RGCNConv(num_features, dim, num_relations=3)
        self.gcs = nn.ModuleList()
        self.num_layers = num_layers
        for i in range(1, num_layers):
            conv = RGCNConv(dim, dim, num_relations=3) 
            self.gcs.append(conv)
        self.lin = Linear(dim, num_classes)

    def forward(self, x, edge_index, edge_type):
        x = F.relu(self.conv1(x, edge_index, edge_type))
        x = F.dropout(x, training=self.training)
        for i in range(1, self.num_layers):
            x = F.relu(self.gcs[i-1](x, edge_index, edge_type))
            x = F.dropout(x, training=self.training)
        return self.lin(x)
    
    
def train(model, data, optimizer):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, data.edge_type)
    mask = data.train_mask
    loss = F.cross_entropy(out[mask], data.y[mask])
    loss.backward()
    optimizer.step()
    return float(loss)


@torch.no_grad()
def test(model, data):
    model.eval()
    pred = model(data.x, data.edge_index, data.edge_type).argmax(dim=-1)

    accs = []
    for split in ['train_mask', 'val_mask']:
        mask = data[split]
        acc = (pred[mask] == data.y[mask]).sum() / mask.sum()
        accs.append(float(acc))
    return accs

In [4]:
def experiment(start,end,test_list, model_type):
    kg_path = lambda graph_name: f'../data/{graph_name}'

    dataset = tl.DNS(root=kg_path('DNS_2m'), start=start, end=end, test_list=test_list, balance_gt=False, domain_file='domains2.csv')
    data = to_homogeneous(dataset.train_data) # training data
    
    model = RGCN(data.x.size(1), dim=64, num_classes=2,num_layers=2)

    if torch.cuda.is_available():
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        torch.cuda.set_device(cuda_device)

        data, model = data.to(device), model.to(device)

    with torch.no_grad():  # Initialize lazy modules.
        out = model(data.x, data.edge_index, data.edge_type)

    optimizer = torch.optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)

    for epoch in range(0, 201):
        loss = train(model, data, optimizer)
        train_acc, val_acc = test(model,data)
        if epoch % 20 == 0:
            print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Train: {train_acc:.4f}, '
                f'Val: {val_acc:.4f}')
        
    model.eval()
    for index, test_data in enumerate(dataset.test_data):
        test_data = to_homogeneous(test_data)
        test_data = test_data.to(device)
        with torch.no_grad():
            pred = model(test_data.x, test_data.edge_index, test_data.edge_type).argmax(dim=-1)
        mask = test_data['val_mask']
        scores = score(pred[mask],test_data.y[mask])
        with open("../results_copy.csv", "a") as logger:
            logger.write("{},{},{},{},".format(model_type,start,end,index))
            logger.write(",".join(str(x) for x in scores.values()))
            logger.write('\n')


        for metric, val in scores.items():
            print(metric, ':{:.4f}'.format(val))
    
for model_type in ['rgcn']:
    for i in range(5):
        experiment(i,i+6,[i+7,i+8], model_type)

Total labeled 897635
Labeled node count for 0, 6: 31778
Labeled node count for 0, 7: 2610
Labeled node count for 0, 8: 2083
Epoch: 000, Loss: 0.7028, Train: 0.7053, Val: 0.6913
Epoch: 020, Loss: 0.4765, Train: 0.7677, Val: 0.7583
Epoch: 040, Loss: 0.4337, Train: 0.8121, Val: 0.8058
Epoch: 060, Loss: 0.4098, Train: 0.8184, Val: 0.8178
Epoch: 080, Loss: 0.3972, Train: 0.8206, Val: 0.8217
Epoch: 100, Loss: 0.3929, Train: 0.8221, Val: 0.8228
Epoch: 120, Loss: 0.3880, Train: 0.8219, Val: 0.8198
Epoch: 140, Loss: 0.3855, Train: 0.8230, Val: 0.8190
Epoch: 160, Loss: 0.3851, Train: 0.8238, Val: 0.8189
Epoch: 180, Loss: 0.3850, Train: 0.8239, Val: 0.8183
Epoch: 200, Loss: 0.3838, Train: 0.8244, Val: 0.8183
tn, fp, fn, tp 1316 289 122 883
acc :0.8425
f1 :0.8442
auc :0.8493
prec :0.7534
recall :0.8786
fpr :0.1801
mi_f1 :0.8425
ma_f1 :0.8381
tn, fp, fn, tp 1069 240 112 662
acc :0.8310
f1 :0.8331
auc :0.8360
prec :0.7339
recall :0.8553
fpr :0.1833
mi_f1 :0.8310
ma_f1 :0.8243
Total labeled 897635
La